# Will the movie be successful on Netflix? - Web Scarping Part
### In this project we will predict whether a new film that will be uploaded to Netflix will be successful movie on Netflix. We will learn about various parameters of successful past films, (such as genre, director, actors, year of publication and our goal column - films with the most liked label indicating the success of the film).

### In this section we will deal with Crawling \ Web Scraping.

### The information was extracted by selenium because Netflix is a very dynamic site and we had to extract the relevant information for us for the project so that we could answer a research question for us from the many options that exist within the site.

###### Note: Because we have a lot of information we will have to run the code for a specific category each time.


In [ ]:
import pandas as pd
import numpy as np
import time
import os
import datetime

# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


### First, we want to automatically connect to Netflix so we can pump out the data.

#### In this section we will automatically connect to the Netflix website, go through movies and select a different category of movie each time.

In [ ]:
# Chromedrive location
ser = Service("C:\\chromedriver\\chromedriver.exe")
driver = webdriver.Chrome(service = ser)

In [ ]:
# Open page by url link
url = "https://www.netflix.com/browse"
driver.get(url)

In [ ]:
# locating inputs for login 
email_box = driver.find_element(By.ID, "id_userLoginId")
password_box = driver.find_element(By.ID,"id_password")
login_button = driver.find_element(By.CLASS_NAME, "btn")

In [ ]:
# login
email_box.send_keys("USER NAME")
password_box.send_keys("PASSWORD")
login_button.click()
time.sleep(3)

In [ ]:
# Choose user
choose_user = driver.find_element(By.CLASS_NAME, "choose-profile").find_elements(By.TAG_NAME,'li')[3]
choose_user.click()
time.sleep(3)

In [ ]:
#Movies catogery
choose_movies = driver.find_element(By.CLASS_NAME,"tabbed-primary-navigation").find_elements(By.TAG_NAME,'li')[3]
choose_movies.click()
time.sleep(5)

In [ ]:
style_menu = driver.find_element(By.CLASS_NAME,"label")
style_menu.click()
time.sleep(4)
click_style = driver.find_elements(By.CLASS_NAME,"sub-menu-item")
click_style[9].click()

time.sleep(4)
grid_view = driver.find_element(By.CLASS_NAME,"aro-grid-toggle")
grid_view.click()


In [ ]:
#Scrollig down the page
check_height = driver.execute_script("return document.body.scrollHeight;") 
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    height = driver.execute_script("return document.body.scrollHeight;") 
    if height == check_height: 
        break 
    check_height = height

driver.find_element(By.TAG_NAME, value ='body').send_keys(Keys.CONTROL + Keys.HOME)
time.sleep(4)

### Second part - create lists so that we can enter the relevant information in them and then we will create an Data frame that contains all the information.

In [ ]:
#basic of topics
movie_titles = [] #1
movie_release_year = [] #2
movie_duration = [] #3
movie_most_liked = []#4
movie_description=[]#5

#dynamic topics
movie_cast= []
movie_genre = []
movie_is = []


movie_director = []
movie_full_cast = []
movie_writer = []
movie_genre_full = []
movie_is_full = []

maturity=[]

In [ ]:
movies = driver.find_elements(By.CLASS_NAME, "boxart-image-in-padded-container")
i=0
for movie in movies:
    if (i % 5 == 0 and i != 0):
        time.sleep(4)
        space_height = 178
        driver.execute_script(f"window.scrollTo(0, {space_height + 97} )") 
    time.sleep(4)
    movie.click()
    time.sleep(5)
    try: # First try
        time.sleep(6)
        try:
            movie_title = driver.find_element(By.CLASS_NAME,value="previewModal--player-titleTreatment-logo").get_attribute('title')
            movie_titles.append(movie_title)
        except:
            movie_titles.append("Nan") #1
        try:
            year = driver.find_element(By.CLASS_NAME,value="year").text 
            movie_release_year.append(year)
        except:
            movie_release_year.append("Nan") #2
       
        try:
            duration = driver.find_element(By.CLASS_NAME,value="duration").text
            movie_duration.append(duration)
        except: 
            movie_duration.append("Nan") #3
        try:
            most_liked = driver.find_element(By.CLASS_NAME,value="css-7yvnxf")
            movie_most_liked.append("1")
              
        except: 
            movie_most_liked.append("0")  #4
               
        try:
            left_con = driver.find_element(By.CLASS_NAME,value="previewModal--detailsMetadata-left")
            description = left_con.find_element(By.CLASS_NAME, value="ptrack-content")
            movie_description.append(description.text)
        except:
            movie_description.append("Nan") #5

        try: # first try
            right_con = driver.find_element(By.CLASS_NAME,value="previewModal--detailsMetadata-right")
            info = right_con.find_elements(By.CLASS_NAME,value="previewModal--tags")
            sentence = []
            for item in info:
                sentence.clear() #clearing to get the next items in line
                sector = item.find_element(By.CLASS_NAME, value="previewModal--tags-label").text
                category = item.find_elements(By.CLASS_NAME, value= "tag-item")
                for cat in category: 
                    sentence.append(cat.text)  
                if(sector == "Cast:"):
                        movie_cast.append(sentence.copy()) #6
                elif (sector == "Genres:"):
                        movie_genre.append(sentence.copy()) #7
                elif (sector == "This movie is:"):
                        movie_is.append(sentence.copy()) #8

        except: # first try
                     print("right con Error")

        try: # second try
            about_con = driver.find_element(By.CLASS_NAME,value="about-container")
            try:
                maturity_number = about_con.find_element(By.CLASS_NAME, value = "maturity-number")
                maturity.append(maturity_number.text)
            except:
                maturity.append("Nan")
            info = about_con.find_elements(By.CLASS_NAME,value="previewModal--tags")
            sentence = []
            for item in info:
                sentence.clear() #clearing to get the next items in line
                sector = item.find_element(By.CLASS_NAME, value="previewModal--tags-label").text      
                category = item.find_elements(By.TAG_NAME, value= "a")
                for cat in category: 
                    sentence.append(cat.text)  
                if (sector == "Genres:"):
                          movie_genre_full.append(sentence.copy()) #9
                if (sector == "Director:"):
                           movie_director.append(sentence.copy())     #10
                if (sector == "Cast:"):
                        movie_full_cast.append(sentence.copy())     #11    
                if (sector == "Writer:"):
                        movie_writer.append(sentence.copy())      #12          
                if (sector == "This movie is:"):
                        movie_is_full.append(sentence.copy()) #13
        except: # second try
                      print("About con Error")
                      
                
    except: # last try
            print("exception")
    if(len(movie_titles) == i):
                movie_titles.append("Nan") #1
    if(len(movie_release_year) == i):
                movie_release_year.append("Nan") #2
    if(len(movie_duration) == i):
                movie_duration.append("Nan")  # 3
    if(len(movie_most_liked) == i):
                movie_most_liked.append("Nan") #4
    if(len(movie_description) == i):
                movie_description.append("Nan") #5
    if(len(movie_cast) == i):
                movie_cast.append("Nan") #6
    if(len(movie_genre) == i):
                movie_genre.append("Nan")  #7
    if(len(movie_is) == i):
                movie_is.append("Nan") #8
    if(len(movie_genre_full) == i):
                movie_genre_full.append("Nan") #9
    if(len(movie_director) == i):
                movie_director.append("Nan") # 10
    if(len(movie_full_cast) == i):
                movie_full_cast.append("Nan") #11
    if(len(movie_writer) == i):
                movie_writer.append("Nan")  #12
    if(len(movie_is_full) == i):
                movie_is_full.append("Nan") #13 
    if(len(maturity) == i):
                maturity.append("Nan")
    time.sleep(4)
    mouse_close = driver.find_element(By.CLASS_NAME,value = "previewModal-close")
    mouse_close.click()
    time.sleep(5)
    i += 1        


In [ ]:
print(len(movie_titles))
print(len(movie_release_year))
print(len(movie_duration))
print(len(movie_most_liked))
print(len(movie_description))
print(len(movie_cast))
print(len(movie_genre))
print(len(movie_is))
print(len(movie_director))
print(len(movie_full_cast))
print(len(movie_writer))
print(len(movie_genre_full))
print(len(movie_is_full))
print(len(maturity))

In [ ]:
df = pd.DataFrame({'Title': movie_titles, 'Year': movie_release_year, 'Duration': movie_duration, 'MostLiked' : movie_most_liked, 
                  'Descrition': movie_description, 'Cast': movie_cast, 'Genre': movie_genre, 'About': movie_is, 'Director': movie_director, "FullCast" : movie_full_cast , "Writer" : movie_writer , "GenreFull" : movie_genre_full , "AboutFull" : movie_is_full , "Age" : maturity })
df.to_csv("Category1.csv")